For some purposes it is preferable to use as few motifs as possible to make
predictions. This notebook uses sparse linear regression to select a subset
of the motifs for each training-validation split used when fitting the
polynomial models. The identities of the columns flagged as significant
by this process are saved to a config file so that the polynomial fitting
code can retrieve these later. In this way, the performance of a polynomial
fitted using a subset of the available motifs can be compared to performance
with the full set.

In [1]:
import os
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLarsIC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
if os.getcwd().endswith("notebooks"):
    os.chdir("..")
    home_dir = os.getcwd()

from gexpress_testing.utilities.utilities import filter_data, cleanup_storage
from gexpress_testing.utilities.utilities import get_tt_split

#Location of the ids of cell lines which are nonredundant -- the only ones we should use
nonred_id_path = os.path.join(home_dir, "config_files", "EpiMapID_Name_nonDup.txt")
#Location where the columns identified as important will be saved. If anything is
#already present there an error will be raised.
output_path = os.path.join(home_dir, "config_files", "lasso_flagged_motifs.py")
if os.path.exists(output_path):
    raise RuntimeError("Output config file already exists; do not overwrite.")

os.mkdir("/scratch/temp")

In [3]:
with open(nonred_id_path, "r", encoding='utf-8') as fhandle:
    nonredundant_ids = [l.split()[0] for l in fhandle]

In [4]:
#Unfortunately the file paths here are hard-coded. TODO: fix this later
pfiles, xfiles, yfiles = filter_data("/stg3/data1/sam/enhancer_prediction/fimo_scan/motif_count_matrices_pro_3",
                                "/stg3/data1/sam/enhancer_prediction/training_y",
                                     None, nonredundant_ids, "/scratch/temp")

No valid enhancer path supplied. Ignoring enhancers. Promoters only will be used.
Now retrieving and sorting files...
Total datapoints: 2129180


In [5]:
tt_splits = [get_tt_split(xfiles, pfiles, yfiles, nonredundant_ids,
                offset = offset) for offset in [0,20,40,60,76]]

The training nonred ids are: ['BSS00004', 'BSS00007', 'BSS00043', 'BSS00045', 'BSS00062', 'BSS00074', 'BSS00079', 'BSS00088', 'BSS00089', 'BSS00096', 'BSS00102', 'BSS00112', 'BSS00113', 'BSS00121', 'BSS00145', 'BSS00160', 'BSS00171', 'BSS00178', 'BSS00189', 'BSS00197', 'BSS00211', 'BSS00214', 'BSS00232', 'BSS00246', 'BSS00281', 'BSS00287', 'BSS00296', 'BSS00310', 'BSS00316', 'BSS00332', 'BSS00352', 'BSS00353', 'BSS00372', 'BSS00376', 'BSS00381', 'BSS00385', 'BSS00439', 'BSS00476', 'BSS00478', 'BSS00481']
The valid nonred ids are: ['BSS00483', 'BSS00505', 'BSS00529', 'BSS00543', 'BSS00556', 'BSS00558', 'BSS00700', 'BSS00708', 'BSS00709', 'BSS00717', 'BSS00720', 'BSS00760', 'BSS00762', 'BSS01065', 'BSS01068', 'BSS01084', 'BSS01110', 'BSS01126', 'BSS01136', 'BSS01155', 'BSS01181', 'BSS01190', 'BSS01208', 'BSS01209', 'BSS01226', 'BSS01261', 'BSS01263', 'BSS01264', 'BSS01267', 'BSS01289', 'BSS01338', 'BSS01344', 'BSS01355', 'BSS01360', 'BSS01366', 'BSS01377', 'BSS01391', 'BSS01393', 'BSS013

In [6]:
#Fit a "sparse" Lasso model to each training split. Find
#the coefficients which are non-negligible -- these are
#features that should be retained -- and save a list of
#these for each split to a config file. The alpha-value
#was determined previously using AIC / BIC with LARS.

significant_columns = []

for i, tt_split in enumerate(tt_splits):
    trainx_files, trainy_files = [], []

    for _, trainfiles in tt_split['train_ids'].items():
        trainx_files += trainfiles['p']
        trainy_files += trainfiles["y"]

    trainx = np.vstack([np.load(x) for x in trainx_files])
    trainy = np.concatenate([np.load(y) for y in trainy_files])
    lasso_model = Lasso(alpha=0.005).fit(trainx, trainy)

    nonnegligible_coefs = np.where(np.abs(lasso_model.coef_.flatten()) > 1e-12)[0]
    significant_columns.append(nonnegligible_coefs)
    print(f"Split {i} complete", flush=True)

Split 0 complete
Split 1 complete
Split 2 complete
Split 3 complete
Split 4 complete


In [7]:
significant_columns = [s.tolist() for s in significant_columns]

In [8]:
with open(output_path, "w+") as fhandle:
    _ = fhandle.write('"""This config file stores a list of positions flagged as\n'
                     'significant for each split by fitting a sparse Lasso model. It is auto-generated\n'
                      'so the formatting is a little clunky."""\n\n')
    _ = fhandle.write(f"KEY_POSITIONS_BY_SPLIT = {significant_columns}")

In [9]:
#Don't forget to remove the temporary files created in scratch!!
shutil.rmtree("/scratch/temp")